# Adversarial Robustness Framework: GTSRB Case Study
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Monthaonos/Advanced_machine_learning/blob/main/main.ipynb)

This interactive notebook demonstrates the adversarial vulnerability and defense mechanisms of deep learning models on the **GTSRB** (German Traffic Sign Recognition Benchmark) dataset.

### Experimental Pipeline:
1. **Phase 1: Training** - Standard and Adversarial (PGD) training for WideResNet/ResNet architectures.
2. **Phase 2: Evaluation** - Quantitative benchmarking of L-infinity robustness.
3. **Phase 3: Universal Patch Analysis** - Targeted L-0 attack using an optimized universal patch.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 🛠️ Environment Initialization
To ensure a reproducible environment, we perform the following steps:
* **Repository Cloning**: Downloads the full source code from GitHub.
* **Dependency Management**: Installs the required scientific stack (PyTorch, Seaborn, TOML).
* **Workspace Preparation**: Sets up the local directory structure for logs and results.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Clone the repository
!git clone https://github.com/Monthaonos/Advanced_machine_learning.git
%cd Advanced_machine_learning

# 2. Install dependencies
!pip install -q -r requirements.txt

# Set visual style
sns.set_theme(style="whitegrid")

## 🚀 Research Pipeline Execution
We execute the `main.py` orchestrator to run the three research phases sequentially. 
The system automatically detects the experimental parameters (prefix, architecture, epsilon) from the `config.toml` file.

* **Phase 1**: Optimizes model weights for both clean and robust variants.
* **Phase 2**: Benchmarks the models against iterative L-infinity attacks.
* **Phase 3**: Optimizes a Universal Patch within the sign's semantic central region (25%-75%).

In [ ]:
# Execute the orchestrator directly via CLI
# This uses your current main.py logic and priorities
!python main.py --train --eval --patch

## 📊 Quantitative Analysis
In this section, we parse the generated CSV reports to visualize the framework's performance. 
We analyze:
* **Training Convergence**: Loss dynamics during Phase 1.
* **Robustness Benchmark**: Model accuracy under L-infinity pressure.
* **Attack Success Rate (ASR)**: The effectiveness of the Universal Patch on the dataset.

In [ ]:
import os
import toml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load configuration to synchronize file discovery
config = toml.load("config.toml")
prefix = config["model"].get("prefix", "test0")
dataset = config["data"]["dataset"]
arch = config["model"]["architecture"]

# 2. Path resolution: Simplified flat structure
# We now target the root folders directly as subdirectories are no longer created.
storage_path = config["project"]["storage_path"].rstrip("/")
results_path = "results"

# --- Phase 1: Training Loss Visualization ---
# Filenames still use the prefix for identification within the flat folder
clean_log = os.path.join(storage_path, f"{prefix}_{dataset}_{arch}_clean_loss.csv")
robust_log = os.path.join(storage_path, f"{prefix}_{dataset}_{arch}_robust_loss.csv")

plt.figure(figsize=(10, 5))
found_logs = False

if os.path.exists(clean_log):
    df_clean = pd.read_csv(clean_log)
    sns.lineplot(data=df_clean, x='epoch', y='loss', label='Standard Training')
    found_logs = True

if os.path.exists(robust_log):
    df_robust = pd.read_csv(robust_log)
    sns.lineplot(data=df_robust, x='epoch', y='loss', label='Adversarial Training')
    found_logs = True

if found_logs:
    plt.title(f"Phase 1: Training Convergence ({prefix})")
    plt.ylabel("Cross-Entropy Loss")
    plt.xlabel("Epoch")
    plt.legend()
    plt.show()
else:
    print(f"[-] Training logs not found in: {storage_path}")

# --- Phase 2: Robustness Benchmarking (L-infinity) ---
# Results are now expected directly in the 'results/' directory
eval_file = os.path.join(results_path, f"{prefix}_{dataset}_{arch}_evaluation_report.csv")

if os.path.exists(eval_file):
    df_eval = pd.read_csv(eval_file)
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(data=df_eval, x='Attack', y='Accuracy', hue='Model', palette='viridis')
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f%%', padding=3)
    plt.title("Phase 2: Model Accuracy under L-infinity Attacks")
    plt.ylabel("Top-1 Accuracy (%)")
    plt.ylim(0, 110)
    plt.show()
else:
    print(f"[-] Evaluation report not found at: {eval_file}")

# --- Phase 3: Universal Patch Success Rate (L-0) ---
patch_file = os.path.join(results_path, f"{prefix}_{dataset}_{arch}_patch_report.csv")

if os.path.exists(patch_file):
    df_patch = pd.read_csv(patch_file)
    plt.figure(figsize=(8, 6))
    ax_asr = sns.barplot(data=df_patch, x='Model', y='ASR', palette='magma')
    for container in ax_asr.containers:
        ax_asr.bar_label(container, fmt='%.2f%%', padding=3)
    plt.title("Phase 3: Universal Patch Attack Success Rate (ASR)")
    plt.ylabel("Success Rate (%)")
    plt.show()
else:
    print(f"[-] Patch report not found at: {patch_file}")

## 🎨 Qualitative Inspection
To conclude the analysis, we visualize adversarial samples generated during Phase 3. 
The image below shows a grid of original signs with the optimized universal patch applied, demonstrating the attack's ability to overlay deceptive features onto traffic signs.

In [ ]:
from IPython.display import Image, display
import os
import toml

# 1. Reload configuration to ensure path consistency
config = toml.load("config.toml")
prefix = config["model"].get("prefix", "test0")
dataset = config["data"]["dataset"]
arch = config["model"]["architecture"]

# 2. Path resolution: Simplified flat structure
# We now target the root 'results' directory directly
results_path = "results"

# 3. Construct the exact filename
# The prefix remains part of the filename for unique identification
viz_filename = f"{prefix}_{dataset}_{arch}_patch_viz.png"
viz_path = os.path.join(results_path, viz_filename)

# 4. Qualitative Visualization Display
# This grid shows original traffic signs vs. patched adversarial versions
if os.path.exists(viz_path):
    print(f"\n🎨 Qualitative Visualization (Prefix: {prefix}):")
    print(f"File source: {viz_path}")
    display(Image(filename=viz_path))
else:
    print(f"[-] Visualization grid not found at: {viz_path}")
    print("[*] Troubleshooting: Ensure the new 'flat' directory structure is active.")